# Locational Marginal Price Map

Reference:

1. Zhang, Q., Li, F. A Dataset for Electricity Market Studies on Western and Northeastern Power Grids in the United States. Sci Data 10, 646 (2023). https://doi.org/10.1038/s41597-023-02448-w
1. https://github.com/enliten/ENLITEN-Grid-Econ-Data/

In [1]:
from tqdm import tqdm

import numpy as np
import pandas as pd
import scipy

import andes
import ams

import pypower.api as pyp

import datetime

import matplotlib
import matplotlib.pyplot as plt

In [2]:
matplotlib.rcdefaults()

In [3]:
%matplotlib inline

In [4]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'andes:{andes.__version__}')
print(f'ams:{ams.__version__}')

Last run time: 2024-03-25 23:04:15
andes:1.9.1.post24+g7a87ad5d
ams:0.9.5.post1+g114d11e


In [5]:
ams.config_logger(stream_level=40)

In [6]:
npcc_lmp_src = scipy.io.loadmat('./data/NPCC_LMP.mat')
npcc_lmp = npcc_lmp_src['LMP_c']
npcc_load_src = scipy.io.loadmat('./data/NPCC_Load.mat')
npcc_load = npcc_load_src['load_dis']

wecc_lmp_src = scipy.io.loadmat('./data/WECC_LMP.mat')
wecc_lmp = wecc_lmp_src['LMP_c']
wecc_load_src = scipy.io.loadmat('./data/WECC_Load.mat')
wecc_load = wecc_load_src['load_dis']

In [7]:
sp_npcc = ams.load('./data/NPCC.m',
                   setup=True,
                   no_output=True,
                   default_config=True,)

In [8]:
total_hour = 24 * 366  # 2020 is a leap year

npcc_mpc = ams.io.matpower.m2mpc('./data/NPCC.m')  # base MPC

total_hour = 100  # check partial hours
for i_hour in tqdm(range(total_hour)):
    npcc_mpc['bus'][:, 2] = npcc_load[:, i_hour]  # update load
    sp = ams.system.System(setup=False, no_output=True, default_config=True)
    ams.io.matpower.mpc2system(npcc_mpc, sp)
    sp.setup()
    sp.DCOPF.run(solver='PIQP')
    if sp.DCOPF.exit_code != 0:
        print(f'Hour {i_hour} DCOPF failed')
    # --- LMP check ---
    lmp_check = np.allclose(a=sp.DCOPF.pi.v/sp.config.mva,
                            b=npcc_lmp[:, i_hour],
                            atol=1e-3)
    if not lmp_check:
        print(f'Hour {i_hour} LMP check failed')
    # --- aBus check ---
    ppc = ams.io.pypower.system2ppc(sp)
    ppopt = pyp.ppoption(VERBOSE=0, OUT_ALL=0, PF_ALG=1, OPF_ALG_DC=200)
    ppc_sol = pyp.rundcopf(ppc, ppopt)
    aBus_check = np.allclose(a=sp.DCOPF.aBus.v * andes.shared.rad2deg,
                             b=ppc_sol['bus'][:, 8],
                             atol=1e-3)
    if not aBus_check:
        print(f'Hour {i_hour} aBus check failed')

100%|██████████| 100/100 [00:14<00:00,  6.98it/s]
